## Non-Graphical EDA

In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
from __future__ import division
%matplotlib inline

In [7]:
vg = pd.read_csv("vgsales.csv")

### Data types

Просмотрим набор типов в данных:

In [8]:
vg.dtypes

Rank              int64
Name             object
Platform         object
Year            float64
Genre            object
Publisher        object
NA_Sales        float64
EU_Sales        float64
JP_Sales        float64
Other_Sales     float64
Global_Sales    float64
dtype: object

Взглянем на данные:

In [9]:
vg.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


На что стоит обратить внимание:

1. Missing variables
2. Наличие временных или координатных переменных
3. Float vs. int переменные
4. Выявление категориальных переменных среди текстовых и числовых
5. Разделение ординальных переменных среди категориальных

Самая простая диагностика распределения - частоты:

In [10]:
vg.Genre.value_counts()

Action          3316
Sports          2346
Misc            1739
Role-Playing    1488
Shooter         1310
Adventure       1286
Racing          1249
Platform         886
Simulation       867
Fighting         848
Strategy         681
Puzzle           582
Name: Genre, dtype: int64

In [11]:
pub_counts = vg.Publisher.value_counts()
pub_counts

Electronic Arts                           1351
Activision                                 975
Namco Bandai Games                         932
Ubisoft                                    921
Konami Digital Entertainment               832
THQ                                        715
Nintendo                                   703
Sony Computer Entertainment                683
Sega                                       639
Take-Two Interactive                       413
Capcom                                     381
Atari                                      363
Tecmo Koei                                 338
Square Enix                                233
Warner Bros. Interactive Entertainment     232
Disney Interactive Studios                 218
Unknown                                    203
Eidos Interactive                          198
Midway Games                               198
505 Games                                  192
Microsoft Game Studios                     189
Acclaim Enter

Зачастую бывает полезно запихнуть все категории с низкими частотами в одну для последующего анализа

In [12]:
vg["New_Publisher"] = vg["Publisher"]
vg.loc[vg["Publisher"].isin(pub_counts[pub_counts<100].index), "New_Publisher"] = "Other"

In [13]:
vg.New_Publisher.value_counts()

Other                                     4487
Electronic Arts                           1351
Activision                                 975
Namco Bandai Games                         932
Ubisoft                                    921
Konami Digital Entertainment               832
THQ                                        715
Nintendo                                   703
Sony Computer Entertainment                683
Sega                                       639
Take-Two Interactive                       413
Capcom                                     381
Atari                                      363
Tecmo Koei                                 338
Square Enix                                233
Warner Bros. Interactive Entertainment     232
Disney Interactive Studios                 218
Unknown                                    203
Eidos Interactive                          198
Midway Games                               198
505 Games                                  192
Microsoft Gam

Бывает полезно создать бинарные переменные для отсутсвующих значений в числовых переменных. Это поможет понять природу возникновения отсутствующих значений

In [14]:
vg["No_year"] = 0
vg.loc[pd.isnull(vg['Year']), "No_year"] = 1
vg["No_year"].sum()

271

Еще одна важная переменная, которую стоит создать, если её еще нет - индекс порядка записей. Её анализ поможет получить дополнительную информацию о том, как данные собирались и позволит избежать некоторых ошибок в анализе

In [15]:
vg["New_index"] = vg.index
vg["New_index"].head()

0    0
1    1
2    2
3    3
4    4
Name: New_index, dtype: int64

Рекомендуется также проверять числовые переменные при помощи таблиц частот - это поможет нам обнаружить смешанные распределения

In [16]:
vg["Global_Sales"].value_counts().head()

0.02    1071
0.03     811
0.04     645
0.05     632
0.01     618
Name: Global_Sales, dtype: int64

Для определения взаимосвязей между категориальными переменными можно строить таблицы сопряженности:

In [17]:
ctab1 = pd.crosstab(vg["New_Publisher"], vg["Genre"])
ctab1

Genre,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
New_Publisher,,,,,,,,,,,,
505 Games,21,7,11,31,6,15,2,10,18,41,27,3
Acclaim Entertainment,14,4,31,5,5,9,44,1,17,0,52,2
Activision,310,25,7,103,60,7,74,41,159,23,144,22
Atari,54,17,37,26,21,22,36,28,40,9,56,17
Capcom,154,22,58,11,46,6,13,38,25,2,3,3
Codemasters,9,6,0,7,0,1,89,3,20,3,10,4
D3Publisher,53,31,3,32,12,11,6,4,12,6,11,3
Deep Silver,27,10,2,7,3,7,7,17,5,17,10,10
Disney Interactive Studios,96,29,0,44,20,3,14,3,3,5,1,0


Для лучшего визуального определения взаимосвязей можно использовать цветовую схему:

In [34]:
from itable import *

In [66]:
resid = (ctab1 - stats.chi2_contingency(ctab1)[3])/np.sqrt(stats.chi2_contingency(ctab1)[3])

pt = PrettyTable(resid.reset_index(), tstyle=TableStyle(theme="theme1"))

cs1 = CellStyle()
cs1.set("background-color", "firebrick")
cs1.set("color", "white")

cs2 = CellStyle()
cs2.set("background-color", "royalblue")
cs2.set("color", "white")

for i in range(resid.shape[0]):
  for j in range(resid.shape[1]):
    if resid.iloc[i,j]>=2:
        pt.set_cell_style(style=cs1, rows=[i], cols=[j+1])
    elif resid.iloc[i,j]<=-2:
        pt.set_cell_style(style=cs2, rows=[i], cols=[j+1])

pt

New_Publisher,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
505 Games,-2.8093615955,-2.04312669416,0.376364973091,2.49593078804,-1.3303646265,3.17891496126,-3.28072707016,-1.74556037598,0.722793998956,9.78864656091,-0.0379786376444,-1.73605979445
Acclaim Entertainment,-3.75973428943,-2.71727014406,7.0371825237,-3.21836740838,-1.54151680182,0.997767483343,8.08271000966,-3.81989113829,0.642037308116,-3.09846472885,5.07998099634,-2.01811167742
Activision,8.22985585888,-5.81735962563,-6.07063147421,0.207151902078,1.09298529427,-4.65612481097,0.0504702747762,-4.97864922594,9.32662068584,-3.90778064385,0.500740626608,-2.84197371495
Atari,-2.18519972914,-2.09938233528,4.27784986023,-1.88801435183,0.363032610495,2.59010284244,1.64524441761,-0.807760633959,2.10786423476,-2.28401591011,0.638508204562,0.549604546152
Capcom,8.90856745916,-1.38583434345,8.72408562965,-4.52816670678,5.68128212276,-2.01823644934,-2.93708721083,0.644354214563,-0.934560663731,-4.01004702009,-6.93814936033,-3.19281091719
Codemasters,-3.88237791911,-1.68435390332,-2.7882982289,-2.20169968266,-2.85023175196,-1.8779215067,22.89362044,-2.88360056873,2.30158950041,-1.75089850847,-2.48517502084,-0.893665293818
D3Publisher,2.66825381736,4.43227796193,-2.08989398207,2.96849880903,0.690671092954,1.78445230432,-2.11576286332,-3.08203670484,-0.668728105659,-1.16202183695,-2.95077727201,-1.65399213007
Deep Silver,0.524782061165,0.176871462229,-1.69739678271,-1.58370938148,-1.37866284938,1.31126739188,-0.726862471431,1.82413192804,-1.49637084592,4.2150036932,-1.75169531912,2.23541913136
Disney Interactive Studios,7.9325561956,2.94434943615,-3.33922494247,4.51255242669,2.4458724635,-1.68314268452,-0.603802104374,-3.74769926143,-3.42953545581,-1.89007608049,-5.37712924024,-2.98934019079
Eidos Interactive,7.05265492762,-2.64117401945,-0.982743450196,-3.42259655534,-2.02343715704,0.396191895042,-0.243139291688,-2.08381453468,4.8880061237,-0.725209855449,-3.78547316294,1.01220100462


Иногда бывает удобно разбить числовые переменные на группы и использовать их в таблицах сопряженности

In [61]:
bins_cut = vg.Global_Sales.quantile(q = np.arange(0,1.1,0.1))
vg["Global_Sales_Cut"] = pd.cut(vg["Global_Sales"], bins = bins_cut)

In [63]:
ctab2 = pd.crosstab(vg["Global_Sales_Cut"], vg["Genre"])
ctab2

Genre,Action,Adventure,Fighting,Misc,Platform,Puzzle,Racing,Role-Playing,Shooter,Simulation,Sports,Strategy
Global_Sales_Cut,,,,,,,,,,,,
"(0.01, 0.02]",187,211,26,99,30,52,71,83,83,70,86,73
"(0.02, 0.05]",370,280,87,229,89,101,140,189,133,123,218,129
"(0.05, 0.08]",294,146,89,174,70,74,108,144,112,98,188,73
"(0.08, 0.12]",303,118,89,190,85,61,128,141,111,75,212,80
"(0.12, 0.17]",305,90,76,160,75,53,110,131,102,59,240,51
"(0.17, 0.25]",335,92,74,201,67,39,122,161,111,74,267,56
"(0.25, 0.38]",352,67,101,165,90,44,123,142,105,104,269,71
"(0.38, 0.61]",386,58,86,180,95,42,127,134,147,92,300,45
"(0.61, 1.21]",341,43,96,142,115,37,142,148,170,77,279,41


Временные переменные бывает удобно переводить в числовые (UNIXTIME) и обрезать до года/месяца/дня

Для оценки взаимосвязи числовых переменных мы можем построить таблицы корреляции:

In [38]:
corr_table = vg.corr()
corr_table

,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,No_year,New_index
Rank,1.000000,0.178814,-0.401362,-0.379123,-0.267785,-0.332986,-0.427407,0.012533,1.000000
Year,0.178814,1.000000,-0.091402,0.006014,-0.169316,0.041058,-0.074735,NaN,0.178811
NA_Sales,-0.401362,-0.091402,1.000000,0.767727,0.449787,0.634737,0.941047,-0.007105,-0.401352
EU_Sales,-0.379123,0.006014,0.767727,1.000000,0.435584,0.726385,0.902836,-0.013860,-0.379113
JP_Sales,-0.267785,-0.169316,0.449787,0.435584,1.000000,0.290186,0.611816,-0.022071,-0.267777
Other_Sales,-0.332986,0.041058,0.634737,0.726385,0.290186,1.000000,0.748331,-0.010802,-0.332978
Global_Sales,-0.427407,-0.074735,0.941047,0.902836,0.611816,0.748331,1.000000,-0.013931,-0.427395
No_year,0.012533,NaN,-0.007105,-0.013860,-0.022071,-0.010802,-0.013931,1.000000,0.012533
New_index,1.000000,0.178811,-0.401352,-0.379113,-0.267777,-0.332978,-0.427395,0.012533,1.000000


Их также можно раскрасить для более простого восприятия:

In [67]:
corr_table2 = PrettyTable(corr_table.reset_index(), tstyle=TableStyle(theme="theme1"))

cs3 = CellStyle()
cs3.set("background-color", "gray")
#cs3.set("color", "gray")


for i in range(corr_table.shape[0]):
  for j in range(corr_table.shape[1]):
    if i == j:
        corr_table2.set_cell_style(style=cs3, rows=[i], cols=[j+1])
    elif corr_table.iloc[i,j]>=0.5:
        corr_table2.set_cell_style(style=cs1, rows=[i], cols=[j+1])
    elif corr_table.iloc[i,j]<=-0.5:
        corr_table2.set_cell_style(style=cs2, rows=[i], cols=[j+1])

corr_table2

index,Rank,Year,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,No_year,New_index
Rank,1.0,0.178813639779,-0.401362246889,-0.379123469093,-0.267784652351,-0.332986424371,-0.427406607989,0.0125329705591,0.999999997939
Year,0.178813639779,1.0,-0.0914021622936,0.00601388738378,-0.169316218391,0.0410576668702,-0.0747347976157,nan,0.178811112247
NA_Sales,-0.401362246889,-0.0914021622936,1.0,0.76772674837,0.44978740707,0.634737271904,0.941047357125,-0.00710485128558,-0.401352331164
EU_Sales,-0.379123469093,0.00601388738378,0.76772674837,1.0,0.435584452809,0.726384889374,0.902835813482,-0.0138600268886,-0.379112926999
JP_Sales,-0.267784652351,-0.169316218391,0.44978740707,0.435584452809,1.0,0.290186249602,0.611815518157,-0.0220712792646,-0.267777134014
Other_Sales,-0.332986424371,0.0410576668702,0.634737271904,0.726384889374,0.290186249602,1.0,0.748330846407,-0.0108023693657,-0.332977659307
Global_Sales,-0.427406607989,-0.0747347976157,0.941047357125,0.902835813482,0.611815518157,0.748330846407,1.0,-0.0139309884549,-0.427395418376
No_year,0.0125329705591,nan,-0.00710485128558,-0.0138600268886,-0.0220712792646,-0.0108023693657,-0.0139309884549,1.0,0.0125326627358
New_index,0.999999997939,0.178811112247,-0.401352331164,-0.379112926999,-0.267777134014,-0.332977659307,-0.427395418376,0.0125326627358,1.0


Таблицы корреляции показывают нам только линейную зависимость, и только между числовыми переменными. Если нас интересует взаимосвясь в более широком смысле, мы можем использовать Mutual Information или другие похожие показатели:

In [17]:
from minepy import MINE
mine = MINE(alpha=0.5, c=15)

vg_for_mi = vg
for j in range(vg_for_mi.shape[1]):
    if vg_for_mi.dtypes[j] == "object":
        vg_for_mi.iloc[:,j] = pd.factorize(vg_for_mi.iloc[:,j])[0]

mi_frame = pd.DataFrame(np.nan, index = vg_for_mi.columns, columns = vg_for_mi.columns)
for j in range(vg_for_mi.shape[1]):
    for k in range(j+1, vg_for_mi.shape[1]):
        mine.compute_score(vg_for_mi.iloc[:,j], vg_for_mi.iloc[:,k])
        mic = np.around(mine.mic(), 3)
        mi_frame.iloc[j,k] = mic

In [18]:
mi_frame

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,New_Publisher,No_year,New_index
Rank,NaN,0.684,0.068,0.063,0.057,0.185,0.586,0.459,0.181,0.576,1.000,0.115,0.047,1.000
Name,NaN,NaN,0.057,0.104,0.106,0.225,0.472,0.363,0.159,0.443,0.686,0.152,0.056,0.684
Platform,NaN,NaN,NaN,0.628,0.063,0.120,0.156,0.185,0.196,0.112,0.067,0.093,0.005,0.068
Year,NaN,NaN,NaN,NaN,0.040,0.221,0.070,0.077,0.092,0.033,0.044,0.091,0.120,0.063
Genre,NaN,NaN,NaN,NaN,NaN,0.116,0.064,0.054,0.065,0.027,0.041,0.088,0.001,0.057
Publisher,NaN,NaN,NaN,NaN,NaN,NaN,0.252,0.220,0.374,0.185,0.186,1.000,0.044,0.185
NA_Sales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.321,0.254,0.416,0.591,0.159,0.007,0.586
EU_Sales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.189,0.492,0.462,0.127,0.004,0.459
JP_Sales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.091,0.183,0.214,0.004,0.181
Other_Sales,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.540,0.129,0.002,0.576


Большинство статистических пакетов могут выводить набор основных статистик по каждой переменной:

In [44]:
vg.describe(include='all')

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,New_Publisher,No_year,New_index
count,16598.000000,16598,16598,16327.000000,16598,16540,16598.000000,16598.000000,16598.000000,16598.000000,16598.000000,16540,16598.000000,16598.000000
unique,NaN,11493,31,NaN,12,578,NaN,NaN,NaN,NaN,NaN,30,NaN,NaN
top,NaN,Need for Speed: Most Wanted,DS,NaN,Action,Electronic Arts,NaN,NaN,NaN,NaN,NaN,Other,NaN,NaN
freq,NaN,12,2163,NaN,3316,1351,NaN,NaN,NaN,NaN,NaN,4487,NaN,NaN
mean,8300.605254,NaN,NaN,2006.406443,NaN,NaN,0.264667,0.146652,0.077782,0.048063,0.537441,NaN,0.016327,8298.500000
std,4791.853933,NaN,NaN,5.828981,NaN,NaN,0.816683,0.505351,0.309291,0.188588,1.555028,NaN,0.126735,4791.574219
min,1.000000,NaN,NaN,1980.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.010000,NaN,0.000000,0.000000
25%,4151.250000,NaN,NaN,2003.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.060000,NaN,0.000000,4149.250000
50%,8300.500000,NaN,NaN,2007.000000,NaN,NaN,0.080000,0.020000,0.000000,0.010000,0.170000,NaN,0.000000,8298.500000
75%,12449.750000,NaN,NaN,2010.000000,NaN,NaN,0.240000,0.110000,0.040000,0.040000,0.470000,NaN,0.000000,12447.750000


Как мы видим, в выведенной таблице не указано количество уникальных значений для числовых переменных. На них стоит посмотреть, что получить представление о разнообразности распределения, и, возможно, найти категориальные переменные

In [47]:
for j in vg.columns:
    print j, vg[j].nunique()    

Rank 16598
Name 11493
Platform 31
Year 39
Genre 12
Publisher 578
NA_Sales 409
EU_Sales 305
JP_Sales 244
Other_Sales 157
Global_Sales 623
New_Publisher 30
No_year 2
New_index 16598


Можно выводить дополнительны показатели коэффициент ассиметрии, коэффициент эксцесса и среднее абсолютное отклонение

$$skew = \frac{\frac{1}{n}\sum(x_i-\bar{x})^3}{s^3} $$

$$kurt = \frac{\frac{1}{n}\sum(x_i-\bar{x})^4}{s^4} $$

In [112]:
vg.Global_Sales.skew()

17.400645097552452

Положительная величина говорит о том масса распределения смещена налево

In [111]:
vg.Global_Sales.kurtosis()

603.93234615959341

In [142]:
vg.Global_Sales.mad()

0.59071007325455005

### Агрегирование

Агрегирование данных позволит нам получить доступ к еще большему объему информации. В случае, если речь идет об аддитивных показателях, например платежах, мы можем использовать меры концентрации, например коэффициент Джини.

In [51]:
def gini(list_of_values):
    sorted_list = sorted(list_of_values)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2.
    return (fair_area - area) / fair_area

In [52]:
Gini_coef = gini(vg.Global_Sales)
Gini_coef

0.71394347734039754